In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

# Download

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import html2text

In [ ]:
h = html2text.HTML2Text(bodywidth=0)
h.ignore_links = True
h.ignore_images = True

def parse_project(project_url):
    with urlopen(project_url) as response:
        soup = BeautifulSoup(response, "html.parser")
    software_header = soup.find("header", {"id": "software-header"})
    title = software_header.find("h1").text.strip()
    info = software_header.find("p").text.strip()
    description = ""
    built_with = ""
    app_details = soup.find("div", {"id": "app-details-left"})
    for d in app_details.find_all("div"):
        if d.get("id") == "built-with":
            built_with = h.handle(str(d)).strip()
        elif not d.get("id"):
            description = h.handle(str(d)).strip()
    return [title, info, description, built_with]


def parse_projects(pages=[1]):
    links = []
    for page in pages:
        with urlopen(f"https://anthropiclondon.devpost.com/project-gallery?page={page}") as response:
            soup = BeautifulSoup(response, "html.parser")
            for e in soup.find_all("a", {"class": "block-wrapper-link fade link-to-software"}):
                links.append(e["href"])
    return links

In [ ]:
projects = parse_projects([1, 2, 3])
len(projects)

In [ ]:
import pandas as pd

data = []
for i, project in enumerate(projects):
    print(i, project)
    data.append([project] + parse_project(project))
df = pd.DataFrame(data=data, columns=["url", "title", "info", "description", "built_with"])
len(df)

In [ ]:
df.to_csv("projects.csv", index=False)

# Evaluate

In [ ]:
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT

anthropic = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))

In [ ]:
role_prompt = f"{HUMAN_PROMPT} You are an independent and objective judge at AI hackathon. To create a new project, all participants followed the hackathon instructions specified in the XML tags <hackathon_instructions></hackathon_instructions>. Your task as the judge is to select the best project according to creteria specified in the XML tags <judge_creteria></judge_creteria>. The projects descriptions specified using Markdown formatted text in the XML tags <project1_overview></project1_overview> and <project2_overview></project2_overview>."
hackathon_instructions = """Here is the hackathon instructions, in <hackathon_instructions></hackathon_instructions> XML tags:
<hackathon_instructions>
The hackathon challenge is to build & innovate with the power of Claude 2. Claude's advanced natural language capabilities enable a huge range of possibilities. You could create an AI assistant to increase government transparency, provide helpful to businesses, or detect misinformation online. Claude's strong reasoning skills also lend themselves well to automating bureaucratic processes to improve efficiency.
Get creative with what you build! It could be a website, app, or voice experience powered by the Claude API. The judges will favor projects that demonstrate technical innovation but also have a clear purpose and benefit.
Build a Claude 2-enabled app that is relevant to Anthropic's mission, useful for startups and businesses, and good for the world.
Winners will be selected based on social impact/future potential, technical implementation, creativity, and pitching quality. Let's use this chance to build AI that makes a positive difference.
The judges will favor projects that demonstrate technical innovation but also have a clear purpose and benefit.
</hackathon_instructions>"""
judging_creteria = """Here is the judging creteria, in <judging_creteria></judging_creteria> XML tags:\n<judging_creteria>
1. Creativity: Is the project's concept innovative and unique? How good is the idea? Does this solve a real problem?
2. Implementation: How well has the team technically implemented the idea?
3. Potential: What is the project's long-term potential for success, growth and impact? Will you keep working on this?
4. Pitch: How effectively does the team present their project? Is this a compelling startup or OSS?
</judging_creteria>"""
judge_instruction = f"Please choose the project that best matches the judging criteria. Output only one project number either #1 or #2. {AI_PROMPT} #"

In [ ]:
import pandas as pd

df = pd.read_csv("projects.csv")
len(df)

In [ ]:
def get_project_overview(project):
    return f"# {project.title}\n\n{project['info']}\n\n{project.description}\n\n{project.built_with}".strip()

In [ ]:
# print(get_project_overview(df.sample(1).iloc[0]))

In [ ]:
import numpy as np

dist = np.zeros((len(df), len(df)))

In [ ]:
results = dict()

In [ ]:
results = pd.read_csv("results.csv", index_col="vs")
results = {k: str(v.score) for k, v in results.iterrows()}

In [ ]:
from tqdm import tqdm

for i in range(len(df)):
    project1_text = get_project_overview(df.iloc[i])
    print(df.iloc[i].title)
    for j in tqdm(range(len(df))):
        project2_text = get_project_overview(df.iloc[j])

        if i == j:
            continue

        pair_key = df.iloc[i].title + " vs " + df.iloc[j].title

        if pair_key in results:
            result = results[pair_key]
        else:

            project1_overview = "Here is the overview of project #1, in <project1_overview></project1_overview> XML tags:\n<project1_overview>" + project1_text + "</project1_overview>"
            project2_overview = "Here is the overview of project #2, in <project2_overview></project2_overview> XML tags:\n<project2_overview>" + project2_text + "</project2_overview>"

            prompt = [
                role_prompt,
                hackathon_instructions,
                project1_overview,
                project2_overview,
                judging_creteria,
                judge_instruction,
            ]
            prompt = "\n\n".join(prompt)

            completion = anthropic.completions.create(
                model="claude-2",
                temperature=0,
                max_tokens_to_sample=1,
                prompt=prompt,
            )
            
            result = completion.completion
        
        if result == "1":
            dist[i, j] = 1
        elif result == "2":
            pass
        else:
            raise ValueError(f"Unexpected result: {result}")
        
        results[pair_key] = result

    pd.DataFrame(results.items(), columns=["vs", "score"]).to_csv("results.csv", index=False)

In [ ]:
dist_final = dist - dist.T

In [ ]:
df["score"] = dist_final.sum(axis=1) / (len(df) - 1)

In [ ]:
df.sort_values("score", ascending=False)[["score", "title", "url"]].to_csv("scores.csv", index=False, float_format="%.2f")